# Benchmarking and optimization

In this notebook, we continue baselining with the NLP problems introduced in chapter 2.

In [ ]:
import os
import time
import random
import numpy as np
import pandas as pd
import email
import re
import nltk
from nltk.corpus import stopwords
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn import metrics
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt

## Enron and fraudulent emails datasets

In [ ]:
enron_filepath = "../data/enron-email-dataset/emails.csv"
# We will preserve the typo in the filename as that is how it appears on Kaggle.
fraud_filepath = "../data/fraudulent-email-corpus/fradulent_emails.txt"

In [ ]:
emails = pd.read_csv(enron_filepath)
emails.shape

In [ ]:
emails.head()

In [ ]:
def extract_messages(df):
    messages = []
    for item in df["message"]:
        e = email.message_from_string(item)
        message_body = e.get_payload()
        messages.append(message_body)
    return messages

In [ ]:
bodies = extract_messages(emails)

In [ ]:
bodies_df = pd.DataFrame(bodies)
bodies_df.head()

In [ ]:
with open(fraud_filepath, "r", encoding="latin1") as infile:
    data = infile.read()
fraud_emails = data.split("From r")
len(fraud_emails)

In [ ]:
fraud_bodies = extract_messages(
    pd.DataFrame(fraud_emails, columns=["message"], dtype=str)
)
fraud_bodies_df = pd.DataFrame(fraud_bodies[1:])
fraud_bodies_df.head()

In [ ]:
print(fraud_bodies_df[0][0])

In [ ]:
Nsamp = 1000
maxtokens = 50
maxtokenlen = 20

In [ ]:
def tokenize(row):
    if row in [None, ""]:
        tokens = ""
    else:
        tokens = str(row).split(" ")[:maxtokens]
    return tokens

In [ ]:
def reg_expressions(row):
    tokens = []
    try:
        for token in row:
            token = token.lower()
            token = re.sub(r"[\W\d]", "", token)
            token = token[:maxtokenlen]
            tokens.append(token)
    except:
        token = ""
        tokens.append(token)
    return tokens

In [ ]:
nltk.download("stopwords")

In [ ]:
english_stopwords = stopwords.words("english")

In [ ]:
def stop_word_removal(row):
    token = [token for token in row if token not in english_stopwords]
    token = filter(None, token)
    return token

In [ ]:
EnronEmails = bodies_df.iloc[:, 0].apply(tokenize)
EnronEmails = EnronEmails.apply(stop_word_removal)
EnronEmails = EnronEmails.apply(reg_expressions)
EnronEmails = EnronEmails.sample(Nsamp)
SpamEmails = fraud_bodies_df.iloc[:, 0].apply(tokenize)
SpamEmails = SpamEmails.apply(stop_word_removal)
SpamEmails = SpamEmails.apply(reg_expressions)
SpamEmails = SpamEmails.sample(Nsamp)
raw_data = pd.concat((SpamEmails, EnronEmails), axis=0).values

In [ ]:
print(raw_data.shape)
print(raw_data[:5])

In [ ]:
Categories = ["spam", "notspam"]
header = [1] * Nsamp + [0] * Nsamp

In [ ]:
def assemble_bag(data):
    used_tokens = []
    all_tokens = []
    for item in data:
        for token in item:
            if token in all_tokens:
                if token not in used_tokens:
                    used_tokens.append(token)
            else:
                all_tokens.append(token)
    df = pd.DataFrame(0, index=np.arange(len(data)), columns=used_tokens)
    for i, item in enumerate(data):
        for token in item:
            if token in used_tokens:
                df.iloc[i][token] += 1
    return df

In [ ]:
EnronSpamBag = assemble_bag(raw_data)
predictors_emails = [column for column in EnronSpamBag.columns]
EnronSpamBag

In [ ]:
def unison_shuffle_data(data, header):
    p = np.random.permutation(len(header))
    data = data[p]
    header = np.asarray(header)[p]
    return data, header

In [ ]:
data, header = unison_shuffle_data(EnronSpamBag.values, header)
idx = int(0.7 * data.shape[0])
train_x_emails = data[:idx]
train_y_emails = header[:idx]
test_x_emails = data[idx:]
test_y_emails = header[idx:]

In [ ]:
clf = RandomForestClassifier(n_jobs=1, random_state=0)
start_time = time.time()
clf.fit(train_x_emails, train_y_emails)
end_time = time.time()
print(f"Training complete in {end_time - start_time}s")
predicted_labels = clf.predict(test_x_emails)
acc_score = accuracy_score(test_y_emails, predicted_labels)
acc_score

In [ ]:
def modelfit(
    alg,
    train_x,
    train_y,
    predictors,
    test_x,
    performCV=True,
    cv_folds=5
):
    alg.fit(train_x, train_y)
    predictions = alg.predict(train_x)
    predprob = alg.predict_proba(train_x)[:, 1]
    if performCV:
        cv_score = cross_val_score(
            alg,
            train_x,
            train_y,
            cv=cv_folds,
            scoring="roc_auc"
        )
    print("Model report")
    print(f"Accuracy: {metrics.accuracy_score(train_y, predictions)}")
    print(f"AUC score (train): {metrics.roc_auc_score(train_y, predprob)}")
    if performCV:
        print(f"CV score: mean: {np.mean(cv_score)} | std: {np.std(cv_score)} "
              f"| min: {np.min(cv_score)} | max: {np.max(cv_score)}")
    feat_imp = pd.Series(
        alg.feature_importances_,
        predictors
    ).sort_values(ascending=False)
    feat_imp[:10].plot(kind="bar", title="Feature importances")
    return alg.predict(test_x), alg.predict_proba(test_x)

In [ ]:
gbm0 = GradientBoostingClassifier(random_state=10)
start_time = time.time()
test_predictions, test_probs = modelfit(
    gbm0,
    train_x_emails,
    train_y_emails,
    predictors_emails,
    test_x_emails
)
end_time = time.time()
print(f"Training complete in {end_time - start_time}s")
predicted_labels = test_predictions
acc_score = accuracy_score(test_y_emails, predicted_labels)
acc_score

In [ ]:
test_probs_max = []
for i in range(test_probs.shape[0]):
    test_probs_max.append(test_probs[i, test_y_emails[i]])
fpr, tpr, thresholds = metrics.roc_curve(test_y_emails, np.array(test_probs_max))
fig, ax = plt.subplots()
plt.plot(fpr, tpr, label="ROC curve")
plt.plot([0, 1], [0, 1], color="navy", linestyle="--")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC curve")
plt.legend(loc="lower right")
plt.show()

## IMDB movie reviews dataset

In [ ]:
def load_data(path):
    data, sentiments = [], []
    for folder, sentiment in (("neg", 0), ("pos", 1)):
        folder = os.path.join(path, folder)
        for name in os.listdir(folder):
            with open(os.path.join(folder, name), "r") as reader:
                text = reader.read()
            text = tokenize(text)
            text = stop_word_removal(text)
            text = reg_expressions(text)
            data.append(text)
            sentiments.append(sentiment)
    data_np = np.array(data)
    data, sentiments = unison_shuffle_data(data_np, sentiments)
    return data, sentiments

In [ ]:
train_path = os.path.join("..", "data", "aclImdb", "train")
raw_data, raw_header = load_data(train_path)

In [ ]:
print(raw_data.shape)
print(len(raw_header))

In [ ]:
random_indices = np.random.choice(
    range(len(raw_header)), size=(Nsamp * 2,),
    replace=False
)
data_train = raw_data[random_indices]
header = raw_header[random_indices]

In [ ]:
# Ensure roughly balanced class distribution.
unique_elements, counts_elements = np.unique(header, return_counts=True)
print(unique_elements)
print(counts_elements)

In [ ]:
MixedBagOfReviews = assemble_bag(data_train)
predictors_imdb = [column for column in MixedBagOfReviews.columns]
MixedBagOfReviews.head()

In [ ]:
data = MixedBagOfReviews.values
idx = int(0.7 * data.shape[0])
train_x_imdb = data[:idx, :]
train_y_imdb = header[:idx]
test_x_imdb = data[idx:, :]
test_y_imdb = header[idx:] 
print(len(train_x_imdb))
print(train_x_imdb)
print(train_y_imdb[:5])
print(len(train_y_imdb))

In [ ]:
clf = RandomForestClassifier(n_jobs=1, random_state=0)
start_time = time.time()
clf.fit(train_x_imdb, train_y_imdb)
end_time = time.time()
print(f"Training complete in {end_time - start_time}s")
predicted_labels = clf.predict(test_x_imdb)
acc_score = accuracy_score(test_y_imdb, predicted_labels)
acc_score

In [ ]:
gbm0 = GradientBoostingClassifier(random_state=10)
start_time = time.time()
test_predictions, test_probs = modelfit(
    gbm0,
    train_x_imdb,
    train_y_imdb,
    predictors_imdb,
    test_x_imdb
)
end_time = time.time()
print(f"Training complete in {end_time - start_time}s")
predicted_labels = test_predictions
acc_score = accuracy_score(test_y_imdb, predicted_labels)
acc_score

In [ ]:
test_probs_max = []
for i in range(test_probs.shape[0]):
    test_probs_max.append(test_probs[i, test_y_imdb[i]])
fpr, tpr, thresholds = metrics.roc_curve(test_y_imdb, np.array(test_probs_max))
fig, ax = plt.subplots()
plt.plot(fpr, tpr, label="ROC curve")
plt.plot([0, 1], [0, 1], color="navy", linestyle="--")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC curve")
plt.legend(loc="lower right")
plt.show()